<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4* Jonatan Rivera

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [45]:
%load_ext tensorboard

In [57]:
import numpy as np
from sklearn.utils import shuffle

#import train_test_split to create training and testing sets
from sklearn.model_selection import train_test_split
def load_quickdraw10(path):
    """ Function returns train and testing matrices w/ corresponding target vectors  for given data
    returns X_train,X_test, y_train,  y_test"""
    path = np.load('quickdraw10.npz')
    data = path
    X = data['arr_0']
    y = data['arr_1']
    X, y = shuffle(X, y)
    
    return train_test_split(X, y, test_size=0.20, random_state=42)


In [58]:
X_train,X_test,y_train,y_test = load_quickdraw10('quickdraw10.npz')

In [59]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((80000, 784), (80000,), (20000, 784), (20000,))

In [29]:
# import numpy and pandas libararies incase we need to modify data in pandas data frame
import numpy
import pandas as pd
#Help with hyperparameter tunning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
#Import model required to build model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf

In [30]:
#Best hyperparameters based on last module
# units 15.000 ; learning rate 0.0010000 ; optimizer adamax; activation function softmax; kernel initializer random_uniform, bias init ones #had accuracy of 63%

In [31]:
# construct model 
model = tf.keras.Sequential(
    
      [Dense(32, activation='relu', input_dim=784),
       Dense(units=15,activation = 'softmax',kernel_initializer = 'random_uniform',bias_initializer = 'ones'),
       Dense(10, activation ='softmax')])


model.compile(
      optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy']
  )
#train model and test it  
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
2094/2094 [==============================] - 1s 652us/step - loss: 1.9224 - accuracy: 0.3622
Epoch 2/5
2094/2094 [==============================] - 1s 602us/step - loss: 1.5284 - accuracy: 0.4556
Epoch 3/5
2094/2094 [==============================] - 1s 597us/step - loss: 1.3870 - accuracy: 0.4771
Epoch 4/5
2094/2094 [==============================] - 1s 593us/step - loss: 1.3058 - accuracy: 0.5081
Epoch 5/5
2094/2094 [==============================] - 1s 602us/step - loss: 1.2404 - accuracy: 0.5356


# Applying Early Stopping Technique

In [37]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import ReLU
import tensorflow as tf
import os

logdir = os.path.join("logs", "EarlyStopping-Loss")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)


model = tf.keras.Sequential(
    
      [Dense(320, activation='relu', input_dim=784),
       Dense(units=150,activation = 'softmax',kernel_initializer = 'random_uniform',bias_initializer = 'ones'),
       Dense(100, activation ='softmax')])


model.compile(
      optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
2094/2094 [==============================] - 5s 2ms/step - loss: 2.9592 - accuracy: 0.2754 - val_loss: 2.1197 - val_accuracy: 0.3012
Epoch 2/99
2094/2094 [==============================] - 4s 2ms/step - loss: 1.9039 - accuracy: 0.2966 - val_loss: 1.7790 - val_accuracy: 0.2985
Epoch 3/99
2094/2094 [==============================] - 4s 2ms/step - loss: 1.7209 - accuracy: 0.3094 - val_loss: 1.6870 - val_accuracy: 0.3119
Epoch 4/99
2094/2094 [==============================] - 4s 2ms/step - loss: 1.6406 - accuracy: 0.3234 - val_loss: 1.6169 - val_accuracy: 0.3212
Epoch 5/99
2094/2094 [==============================] - 4s 2ms/step - loss: 1.5697 - accuracy: 0.3469 - val_loss: 1.5502 - val_accuracy: 0.3546
Epoch 6/99
2094/2094 [==============================] - 4s 2ms/step - loss: 1.5108 - accuracy: 0.3804 - val_loss: 1.5015 - val_accuracy: 0.4049
Epoch 7/99
2094/2094 [==============================] - 4s 2ms/step - loss: 1.4647 - accuracy: 0.4169 - val_loss: 1.4623 - val_accuracy:

# Applying Weight Decay Technique


In [76]:
from tensorflow.keras import regularizers

logdir = os.path.join("logs", "EarlyStopping+L2_WeightDecay")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)



model = tf.keras.Sequential(
    
      [Dense(512, activation='relu', input_dim=784,kernel_regularizer=regularizers.l2(0.01) ),
       ReLU(negative_slope=.01),
       Dense(units=512,activation = 'softmax', kernel_regularizer=regularizers.l2(0.01)),
       ReLU(negative_slope=.01),
       Dense(512,kernel_regularizer=regularizers.l2(0.01), activation ='softmax')])



model.compile(
      optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
2500/2500 [==============================] - 14s 6ms/step - loss: 4.6616 - accuracy: 0.1077 - val_loss: 2.9412 - val_accuracy: 0.0964
Epoch 2/99
2500/2500 [==============================] - 17s 7ms/step - loss: 2.5818 - accuracy: 0.0985 - val_loss: 2.4158 - val_accuracy: 0.0975
Epoch 3/99
2500/2500 [==============================] - 20s 8ms/step - loss: 2.3466 - accuracy: 0.1477 - val_loss: 2.2344 - val_accuracy: 0.1938
Epoch 4/99
2500/2500 [==============================] - 21s 8ms/step - loss: 2.1594 - accuracy: 0.1954 - val_loss: 2.0987 - val_accuracy: 0.1966
Epoch 5/99
2500/2500 [==============================] - 17s 7ms/step - loss: 2.0838 - accuracy: 0.1977 - val_loss: 2.0670 - val_accuracy: 0.1973
Epoch 6/99
2500/2500 [==============================] - 16s 6ms/step - loss: 2.0648 - accuracy: 0.1987 - val_loss: 2.0639 - val_accuracy: 0.1951
Epoch 7/99
2500/2500 [==============================] - 18s 7ms/step - loss: 2.0587 - accuracy: 0.1982 - val_loss: 2.0584 - val_ac

In [43]:
model.summary()


Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_51 (Dense)             (None, 512)               401920    
_________________________________________________________________
re_lu_12 (ReLU)              (None, 512)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 512)               262656    
_________________________________________________________________
re_lu_13 (ReLU)              (None, 512)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 512)               262656    
Total params: 927,232
Trainable params: 927,232
Non-trainable params: 0
_________________________________________________________________


In [46]:
%tensorboard --logdir logs

In [70]:
from tensorflow.keras.layers import Dropout

from tensorflow.keras.constraints import MaxNorm

wc = MaxNorm(max_value=2)

logdir = os.path.join("logs", "EarlyStopping+WeightConstraint+Dropout")

tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)

model = tf.keras.Sequential(
    
      [Dense(320, activation='relu', input_dim=784),
       Dense(units=150,activation = 'softmax',kernel_initializer = 'random_uniform',bias_initializer = 'ones'),
       Dense(100, activation ='softmax')])

model.compile(
      optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])


model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[tensorboard_callback, stop])

Epoch 1/99
2500/2500 [==============================] - 10s 4ms/step - loss: 2.7336 - accuracy: 0.2132 - val_loss: 2.0452 - val_accuracy: 0.2544
Epoch 2/99
2500/2500 [==============================] - 10s 4ms/step - loss: 1.9706 - accuracy: 0.2487 - val_loss: 2.0956 - val_accuracy: 0.1902
Epoch 3/99
2500/2500 [==============================] - 10s 4ms/step - loss: 1.8425 - accuracy: 0.2656 - val_loss: 1.9536 - val_accuracy: 0.2508
Epoch 4/99
2500/2500 [==============================] - 10s 4ms/step - loss: 1.8018 - accuracy: 0.2710 - val_loss: 1.7503 - val_accuracy: 0.2779
Epoch 5/99
2500/2500 [==============================] - 10s 4ms/step - loss: 1.7509 - accuracy: 0.2771 - val_loss: 1.7336 - val_accuracy: 0.2730
Epoch 6/99
2500/2500 [==============================] - 10s 4ms/step - loss: 1.7305 - accuracy: 0.2864 - val_loss: 1.7104 - val_accuracy: 0.2868
Epoch 7/99
2500/2500 [==============================] - 12s 5ms/step - loss: 1.7519 - accuracy: 0.2776 - val_loss: 1.6934 - val_ac

In [71]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6007 (pid 15324), started 0:45:53 ago. (Use '!kill 15324' to kill it.)

# Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [77]:
import tensorflow as tf
stop = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3)
cpoint = tf.keras.callbacks.ModelCheckpoint("weights_best.h5",
                                            verbose=1, 
                                            save_weights_only=True)

def create_model():
    model = tf.keras.Sequential([
        Dense(320, activation='relu', input_dim=784),
        Dense(units=150,activation = 'softmax',kernel_initializer = 'random_uniform',bias_initializer = 'ones'),
        Dense(100, activation ='softmax')])
  

    model.compile(
      optimizer= tf.keras.optimizers.Adamax(learning_rate=0.001),
      loss='sparse_categorical_crossentropy',
      metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=99, 
          validation_data=(X_test,y_test),
          callbacks=[cpoint, stop])
    

    return model

model = create_model()



Epoch 1/99
2493/2500 [============================>.] - ETA: 0s - loss: 2.8940 - accuracy: 0.3263
Epoch 00001: saving model to weights_best.h5
2500/2500 [==============================] - 6s 2ms/step - loss: 2.8916 - accuracy: 0.3263 - val_loss: 2.0195 - val_accuracy: 0.3535
Epoch 2/99
2490/2500 [============================>.] - ETA: 0s - loss: 1.8286 - accuracy: 0.3487
Epoch 00002: saving model to weights_best.h5
2500/2500 [==============================] - 6s 2ms/step - loss: 1.8284 - accuracy: 0.3487 - val_loss: 1.7065 - val_accuracy: 0.3540
Epoch 3/99
2470/2500 [============================>.] - ETA: 0s - loss: 1.6526 - accuracy: 0.3567
Epoch 00003: saving model to weights_best.h5
2500/2500 [==============================] - 5s 2ms/step - loss: 1.6523 - accuracy: 0.3568 - val_loss: 1.6393 - val_accuracy: 0.3491
Epoch 4/99
2479/2500 [============================>.] - ETA: 0s - loss: 1.5888 - accuracy: 0.3598
Epoch 00004: saving model to weights_best.h5
2500/2500 [==================

In [78]:
model.fit(X_train, y_train, epochs=2, 
          validation_data=(X_test,y_test),
          verbose=2,
          callbacks=[cpoint])

Epoch 1/2

Epoch 00001: saving model to weights_best.h5
2500/2500 - 4s - loss: 0.8888 - accuracy: 0.6315 - val_loss: 0.9249 - val_accuracy: 0.6319
Epoch 2/2

Epoch 00002: saving model to weights_best.h5
2500/2500 - 4s - loss: 0.8763 - accuracy: 0.6333 - val_loss: 0.9266 - val_accuracy: 0.6273


In [79]:
model.evaluate(X_test, y_test)

625/625 [==============================] - 1s 878us/step - loss: 0.9266 - accuracy: 0.6273


[0.9265751838684082, 0.6273000240325928]

# Load Exported Model

In [80]:
m = create_model()  # Start with same architecture
m.load_weights('./weights_best.h5')  # Load instead of train
m.summary()

Epoch 1/99
2490/2500 [============================>.] - ETA: 0s - loss: 2.8439 - accuracy: 0.2732
Epoch 00001: saving model to weights_best.h5
2500/2500 [==============================] - 4s 2ms/step - loss: 2.8405 - accuracy: 0.2731 - val_loss: 1.9791 - val_accuracy: 0.2843
Epoch 2/99
2483/2500 [============================>.] - ETA: 0s - loss: 1.8144 - accuracy: 0.2851
Epoch 00002: saving model to weights_best.h5
2500/2500 [==============================] - 4s 2ms/step - loss: 1.8139 - accuracy: 0.2850 - val_loss: 1.7640 - val_accuracy: 0.2840
Epoch 3/99
2480/2500 [============================>.] - ETA: 0s - loss: 1.7023 - accuracy: 0.2880
Epoch 00003: saving model to weights_best.h5
2500/2500 [==============================] - 4s 2ms/step - loss: 1.7020 - accuracy: 0.2879 - val_loss: 1.7430 - val_accuracy: 0.2823
Epoch 4/99
2499/2500 [============================>.] - ETA: 0s - loss: 1.6706 - accuracy: 0.2874
Epoch 00004: saving model to weights_best.h5
2500/2500 [==================

In [81]:
m.evaluate(X_test, y_test)

625/625 [==============================] - 0s 800us/step - loss: 1.3394 - accuracy: 0.3983


[1.3393654823303223, 0.3982999920845032]

In [82]:
# Save entire model
# i.e. both weights and architecture
# Create and train a new model instance.
model = create_model()
model.fit(X_train, y_train, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/99
2500/2500 [==============================] - ETA: 0s - loss: 2.9471 - accuracy: 0.1052
Epoch 00001: saving model to weights_best.h5
2500/2500 [==============================] - 4s 2ms/step - loss: 2.9471 - accuracy: 0.1052 - val_loss: 2.3615 - val_accuracy: 0.0975
Epoch 2/99
2468/2500 [============================>.] - ETA: 0s - loss: 2.3227 - accuracy: 0.1004
Epoch 00002: saving model to weights_best.h5
2500/2500 [==============================] - 4s 2ms/step - loss: 2.3225 - accuracy: 0.1004 - val_loss: 2.3070 - val_accuracy: 0.0964
Epoch 3/99
2479/2500 [============================>.] - ETA: 0s - loss: 2.2392 - accuracy: 0.1540
Epoch 00003: saving model to weights_best.h5
2500/2500 [==============================] - 4s 2ms/step - loss: 2.2381 - accuracy: 0.1546 - val_loss: 2.1135 - val_accuracy: 0.1873
Epoch 4/99
2468/2500 [============================>.] - ETA: 0s - loss: 2.0537 - accuracy: 0.1901
Epoch 00004: saving model to weights_best.h5
2500/2500 [==================

In [84]:
# Load it back fresh, including weights and architecture
new_model = tf.keras.models.load_model('saved_model/my_model')
# Recompiling the model tells it how to calculate accuracy correctly :)
new_model.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_84 (Dense)             (None, 320)               251200    
_________________________________________________________________
dense_85 (Dense)             (None, 150)               48150     
_________________________________________________________________
dense_86 (Dense)             (None, 100)               15100     
Total params: 314,450
Trainable params: 314,450
Non-trainable params: 0
_________________________________________________________________


In [85]:
# Saving the "full model" does save the architecture and weights
# but not the compile details (since those aren't really needed for inference)
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam',
                  metrics=['accuracy'])

In [86]:
new_model.evaluate(X_test, y_test)

625/625 [==============================] - 1s 854us/step - loss: 0.9319 - accuracy: 0.6334


[0.931868851184845, 0.633400022983551]

In [87]:
!ls -alh  # checking memory of model, 96B

total 54152
drwxr-xr-x@ 11 johnrivera  staff   352B Sep 10 16:27 .
drwxr-xr-x@ 15 johnrivera  staff   480B Sep 10 16:26 ..
drwxr-xr-x@  5 johnrivera  staff   160B Sep 10 15:25 .ipynb_checkpoints
-rw-r--r--   1 johnrivera  staff    49K Sep 10 14:08 9_10_2020.ipynb
-rw-r--r--@  1 johnrivera  staff    51K Sep 10 16:25 9_10_v2.ipynb
-rw-r--r--   1 johnrivera  staff    67K Sep 10 16:27 LS_DS_434_Deploy_Assignment.ipynb
drwxr-xr-x@  4 johnrivera  staff   128B Sep 10 13:02 Past_Lect
drwxr-xr-x@  5 johnrivera  staff   160B Sep 10 16:14 logs
-rw-r--r--   1 johnrivera  staff    24M Sep  6 18:52 quickdraw10.npz
drwxr-xr-x@  3 johnrivera  staff    96B Sep 10 16:25 saved_model
-rw-r--r--   1 johnrivera  staff   1.2M Sep 10 16:22 weights_best.h5


### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 